In [1]:
!pip install cryptography


In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os

In [3]:
# Set up password-based encryption (generate a key from a password)
password = b'securepassword'
salt = os.urandom(16)  # A random salt for each encryption session

In [4]:

# Derive an AES key from the password
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)

In [5]:
key = kdf.derive(password)

In [6]:
# Encryption parameters
iv = os.urandom(16)  # Initialization vector

In [7]:
# Sample data to encrypt (at rest)
data = b"This is sensitive data to be encrypted at rest."

In [8]:
# Set up AES encryption
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()

In [9]:

# Pad data to match block size
padder = padding.PKCS7(algorithms.AES.block_size).padder()
padded_data = padder.update(data) + padder.finalize()

In [10]:
# Encrypt the data
encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

In [14]:

# Save the encrypted data to a file
with open("encrypted_data.bin", "wb") as f:
    f.write(salt + iv + encrypted_data)


In [13]:
# Print the encrypted message in hexadecimal format
print("Encrypted message (in hexadecimal):", encrypted_data.hex())

Encrypted message (in hexadecimal): 042aeb20b5f5550a2a32ed0fced6761d2734f79e5ad3f7ff279fe0e1eff4b126db8c7eab4eec8f1ada391f61f4f5ce10


In [ ]:
# Read the encrypted data
with open("encrypted_data.bin", "rb") as f:
    salt = f.read(16)   # First 16 bytes are the salt
    iv = f.read(16)      # Next 16 bytes are the IV
    encrypted_data = f.read()  # Remaining bytes are the encrypted data

# Derive the key again from the password and salt
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)
key = kdf.derive(password)

# Decrypt
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
decryptor = cipher.decryptor()
padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

# Remove padding
unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
data = unpadder.update(padded_data) + unpadder.finalize()

print("Decrypted data:", data.decode())


In [12]:
# Read the encrypted data
with open("encrypted_data.bin", "rb") as f:
    salt = f.read(16)   # First 16 bytes are the salt
    iv = f.read(16)      # Next 16 bytes are the IV
    encrypted_data = f.read()  # Remaining bytes are the encrypted data

In [15]:
# Derive the key again from the password and salt
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)
key = kdf.derive(password)

In [20]:

# Decrypt
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
decryptor = cipher.decryptor()
padded_data = decryptor.update(encrypted_data) + decryptor.finalize()
# Print the decrypted message


Decrypted message: This is sensitive data to be encrypted at rest.
